# Position of the Sun

### Input variables - Latitude, Longitude, Date, Time, Timezone
### Output variables - Azimuth, Elevation

In [64]:
# sunpos.py
import math
def sunpos(when, location, refraction):
# Extract the passed data
    year, month, day, hour, minute, second, timezone = when
    latitude, longitude = location
# Math typing shortcuts
    rad, deg = math.radians, math.degrees
    sin, cos, tan = math.sin, math.cos, math.tan
    asin, atan2 = math.asin, math.atan2
# Convert latitude and longitude to radians
    rlat = rad(latitude)
    rlon = rad(longitude)
# Decimal hour of the day at Greenwich
    greenwichtime = hour - timezone + minute / 60 + second / 3600
# Days from J2000, accurate from 1901 to 2099
    daynum = (
        367 * year
        - 7 * (year + (month + 9) // 12) // 4
        + 275 * month // 9
        + day
        - 730531.5
        + greenwichtime / 24
    )
# Mean longitude of the sun
    mean_long = daynum * 0.01720279239 + 4.894967873
# Mean anomaly of the Sun
    mean_anom = daynum * 0.01720197034 + 6.240040768
# Ecliptic longitude of the sun
    eclip_long = (
        mean_long
        + 0.03342305518 * sin(mean_anom)
        + 0.0003490658504 * sin(2 * mean_anom)
    )
# Obliquity of the ecliptic
    obliquity = 0.4090877234 - 0.000000006981317008 * daynum
# Right ascension of the sun
    rasc = atan2(cos(obliquity) * sin(eclip_long), cos(eclip_long))
# Declination of the sun
    decl = asin(sin(obliquity) * sin(eclip_long))
# Local sidereal time
    sidereal = 4.894961213 + 6.300388099 * daynum + rlon
# Hour angle of the sun
    hour_ang = sidereal - rasc
# Local elevation of the sun
    elevation = asin(sin(decl) * sin(rlat) + cos(decl) * cos(rlat) * cos(hour_ang))
# Local azimuth of the sun
    azimuth = atan2(
        -cos(decl) * cos(rlat) * sin(hour_ang),
        sin(decl) - sin(rlat) * sin(elevation),
    )
# Convert azimuth and elevation to degrees
    azimuth = into_range(deg(azimuth), 0, 360)
    elevation = into_range(deg(elevation), -180, 180)
# Refraction correction (optional)
    if refraction:
        targ = rad((elevation + (10.3 / (elevation + 5.11))))
        elevation += (1.02 / tan(targ)) / 60
# Return azimuth and elevation in degrees
    return (round(azimuth, 2), round(elevation, 2))
def into_range(x, range_min, range_max):
    shiftedx = x - range_min
    delta = range_max - range_min
    return (((shiftedx % delta) + delta) % delta) + range_min
if __name__ == "__main__":
# Close Encounters latitude, longitude
    location = (40.602778, -104.741667)
# Fourth of July, 2022 at 11:20 am MDT (-6 hours)
    when = (2022, 7, 4, 11, 20, 0, -6)
# Get the Sun's apparent location in the sky
    azimuth, elevation = sunpos(when, location, True)
# Output the results
    print("\nWhen: ", when)
    print("Where: ", location)
    print("Azimuth: ", azimuth)
    print("Elevation: ", elevation)
# When:  (2022, 7, 4, 11, 20, 0, -6)
# Where:  (40.602778, -104.741667)
# Azimuth:  121.38
# Elevation:  61.91


When:  (2022, 7, 4, 11, 20, 0, -6)
Where:  (40.602778, -104.741667)
Azimuth:  121.38
Elevation:  61.91


In [65]:
if __name__ == "__main__":
# Close Encounters latitude, longitude
    location = (53.82, -2.24)
# Fourth of July, 2022 at 11:20 am MDT (-6 hours)
    when = (2022, 7, 4, 11, 20, 0, 0)
# Get the Sun's apparent location in the sky
    azimuth, elevation = sunpos(when, location, True)
# Output the results
    print("\nWhen: ", when)
    print("Where: ", location)
    print("Azimuth: ", azimuth)
    print("Elevation: ", elevation)


When:  (2022, 7, 4, 11, 20, 0, 0)
Where:  (53.82, -2.24)
Azimuth:  156.72
Elevation:  57.44


### Upload Cities data in the UK

#### ML - Training dataset

In [83]:
import pandas as pd

df_cities = pd.read_csv('UK Cities Location.csv')

df_cities.head()

,City,Latitude,Longitude
0,York,53.958332,-1.080278
1,Worcester,52.192001,-2.220000
2,Winchester,51.063202,-1.308000
3,Wells,51.209000,-2.647000
4,Wakefield,53.680000,-1.490000


#### ML - Testing dataset

In [81]:
import pandas as pd

df_world = pd.read_csv('worldcities.csv')

df_world.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


### Import DateTime

In [60]:
import datetime

now = datetime.datetime.today()

print(now)

2021-04-22 17:13:49.587210


### Azimuth & Elevation for all Cities in UK

#### Machine learning Training dataset - UK cities

In [87]:
for index, row in df_cities.iterrows():
    azimuth, elevation = sunpos((now.year, now.month, now.day, now.hour, now.minute, now.second, 0), 
                                (row['Latitude'], row['Longitude']), 
                                True)
    print(index, row['City'], azimuth, elevation)

0 York 267.56 17.27
1 Worcester 267.2 18.03
2 Winchester 268.29 17.51
3 Wells 267.19 18.34
4 Wakefield 267.32 17.52
5 Truro 265.62 19.92
6 Sunderland 267.02 17.4
7 Sheffield 267.43 17.52
8 Salford 266.73 18.01
9 St. Davids 264.86 19.92
10 St.Albans 268.84 16.89
11 Ripon 267.15 17.52
12 Portsmouth 268.54 17.38
13 Perth 264.82 18.45
14 Nottingham 267.82 17.35
15 Newry 263.14 20.32
16 Newcastle Upon Tyne 266.82 17.52
17 Liverpool 266.19 18.42
18 Lincoln 268.23 16.97
19 Lichfield 267.36 17.77
20 Leicester 267.93 17.35
21 Lancaster 266.13 18.27
22 Hereford 266.85 18.34
23 Gloucester 267.29 18.06
24 Glasgow 264.31 18.95
25 Exeter 266.65 18.92
26 Ely 269.12 16.51
27 Durham 266.9 17.52
28 Dundee 265.19 18.18
29 Derry 262.03 20.77
30 Derby 267.58 17.54
31 Coventry 267.7 17.59
32 Chichester 268.77 17.18
33 Chester 266.34 18.38
34 Chelmsford 269.49 16.39
35 Carlisle 265.73 18.29
36 Canterbury 270.09 16.02
37 Cambridge 269.06 16.6
38 Brighton & Hove 269.26 16.79
39 Bradford 267.07 17.67
40 Bath 26

### Azimuth & Elevation for all Cities in the World

#### Machine learning Testing dataset - World cities

In [90]:
for index, row in df_world.head(300).iterrows():
    azimuth, elevation = sunpos((now.year, now.month, now.day, now.hour, now.minute, now.second, 0), 
                                (row['lat'], row['lng']), 
                                True)
    print(index, row['city'], azimuth, elevation)

0 Tokyo 44.43 -29.68
1 Jakarta 41.62 -81.62
2 Delhi 327.31 -42.86
3 Mumbai 316.2 -48.0
4 Manila 36.97 -56.6
5 Shanghai 27.25 -42.25
6 São Paulo 315.39 42.19
7 Seoul 31.06 -34.43
8 Mexico City 106.7 69.3
9 Guangzhou 19.66 -52.52
10 Beijing 18.44 -35.8
11 Cairo 291.13 -10.58
12 New York 190.08 61.43
13 Kolkāta 338.99 -52.84
14 Moscow 298.82 -3.6
15 Bangkok 358.59 -63.81
16 Buenos Aires 333.9 39.13
17 Shenzhen 21.14 -52.82
18 Dhaka 342.64 -52.36
19 Lagos 281.64 9.02
20 Istanbul 291.09 -4.79
21 Ōsaka 40.9 -32.73
22 Karachi 314.32 -40.02
23 Bangalore 316.47 -55.49
24 Tehran 306.09 -22.79
25 Kinshasa 282.14 -4.98
26 Ho Chi Minh City 13.31 -66.12
27 Los Angeles 111.41 48.1
28 Rio de Janeiro 311.71 40.34
29 Nanyang 15.34 -43.3
30 Chennai 320.11 -57.15
31 Chengdu 4.16 -46.82
32 Lahore 325.45 -39.07
33 Paris 271.73 15.16
34 London 269.08 16.77
35 Linyi 22.06 -39.8
36 Tianjin 19.57 -36.31
37 Shijiazhuang 16.63 -38.01
38 Baoding 17.61 -37.0
39 Zhoukou 17.9 -42.19
40 Lima 355.74 65.45
41 Hyderābād 

### Validating train data and test data results - Validation complete as of Apr 22 2021 - 17:53 


#### Training dataset  - City = London --> 269.08 16.76 ; 
#### Testing dataset    -  City = London --> 269.08 16.77  ;